<a href="https://colab.research.google.com/github/KoganTheDev/cloud-computing-project/blob/main/updated_cloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install firebase
import ipywidgets as widgets
from IPython.display import display, clear_output
import random
import time
import matplotlib.pyplot as plt
import io
from IPython.display import display as ipy_display
from typing import Dict, List, Callable
from firebase import firebase
from datetime import datetime
import matplotlib.dates as mdates
from zoneinfo import ZoneInfo
local_tz = ZoneInfo("Asia/Jerusalem")
# --- Constants ---
BLUE_COLOR = '#3498db'
DARKER_BLUE_COLOR = '#2c3e50'
ORANGE_COLOR = '#FF851B'
LIGHT_BLUE_COLOR = '#f0f8ff'
LIGHTER_BLUE_COLOR = '#f7f9fc'
WHITE_COLOR = 'rgba(255,255,255,0.7)'
LIGHT_GREY_BORDER = '#ddd'
BOX_SHADOW = '0px 4px 8px rgba(0,0,0,0.2)'
BORDER_RADIUS = '12px'
PADDING = '30px'
MARGIN = '20px'

# --- Data Types ---
SensorData = Dict[str, Dict[str, float or str]]
EngineersPerformance = Dict[str, Dict[str, int]]
HistoricalSensorData = Dict[str, Dict[str, List[float] or List[str]]]

# --- Firebase Setup ---
FBconn = firebase.FirebaseApplication(
    'https://cloudteamwolf-default-rtdb.europe-west1.firebasedatabase.app', None
)
local_tz = ZoneInfo("Asia/Jerusalem")
DATABASE_PATH = 'myTest1'
INDOOR_SENSOR_PATH = f'{DATABASE_PATH}/indoor'
OUTDOOR_SENSOR_PATH = f'{DATABASE_PATH}/outdoor'

# --- Dummy Data (Replace with your actual data) ---
daily_tasks: List[str] = [
    "Review production logs for anomalies",
    "Check calibration of robotic arms",
    "Optimize energy consumption routines",
    "Inspect conveyor belt system",
]

engineers_performance: EngineersPerformance = {
    "Alice": {"Points": 1550, "Improvements Completed": 32},
    "Bob": {"Points": 1200, "Improvements Completed": 25},
    "Charlie": {"Points": 1800, "Improvements Completed": 40},
    "David": {"Points": 950, "Improvements Completed": 18},
}


# --- Helper Functions ---

def create_html_widget(value: str, **layout_kwargs) -> widgets.HTML:
    """Creates an HTML widget with specified layout."""
    layout = widgets.Layout(**layout_kwargs)
    return widgets.HTML(value=value, layout=layout)

def create_button(description: str, button_style: str = 'primary', **layout_kwargs) -> widgets.Button:
    """Creates a Button widget with specified layout."""
    default_layout_kwargs = {
        'width': '200px',
        'height': '60px',
        'border_radius': BORDER_RADIUS,
        'font_size': '16px'
    }
    default_layout_kwargs.update(layout_kwargs) # Override defaults
    layout = widgets.Layout(**default_layout_kwargs)
    return widgets.Button(description=description, button_style=button_style, layout=layout)

def create_text_input(description: str, **layout_kwargs) -> widgets.Text:
    """Creates a Text input widget with specified layout."""
    default_layout_kwargs = {
        'width': '600px',
        'height': '40px',
        'border_radius': BORDER_RADIUS,
        'padding': '10px',
        'font_size': '16px'
    }
    default_layout_kwargs.update(layout_kwargs)
    layout = widgets.Layout(**default_layout_kwargs)
    return widgets.Text(description=description, layout=widgets.Layout(**default_layout_kwargs))

def create_output(**layout_kwargs) -> widgets.Output:
    """Creates an Output widget with specified layout."""
    layout = widgets.Layout(**layout_kwargs)
    return widgets.Output(layout=layout)

def create_dropdown(options: List[str], description: str, **layout_kwargs) -> widgets.Dropdown:
    """Creates a Dropdown widget."""
    default_layout_kwargs = {'width': '300px', 'margin': '10px 0'}
    default_layout_kwargs.update(layout_kwargs)
    layout = widgets.Layout(**default_layout_kwargs)
    return widgets.Dropdown(options=options, description=description, layout=layout)

def create_vbox(children: List[widgets.Widget], **layout_kwargs) -> widgets.VBox:
    """Creates a VBox container with specified layout."""
    default_layout_kwargs = {
        'padding': PADDING,
        'border': f'2px solid {BLUE_COLOR}',
        'border_radius': '15px',
        'margin': MARGIN,
        'background_color': LIGHT_BLUE_COLOR
    }
    default_layout_kwargs.update(layout_kwargs)
    layout = widgets.Layout(**default_layout_kwargs)
    return widgets.VBox(children=children, layout=layout)

def create_hbox(children: List[widgets.Widget], **layout_kwargs) -> widgets.HBox:
    """Creates an HBox container"""
    layout = widgets.Layout(**layout_kwargs)
    return widgets.HBox(children = children, layout=layout)

def perform_search(query: str) -> List[str]:
    """Searches all historical sensor data for matching keys or values."""
    query_lower = query.lower()
    results = []

    # Fetch complete indoor and outdoor data
    indoor_data = FBconn.get(INDOOR_SENSOR_PATH, None)
    outdoor_data = FBconn.get(OUTDOOR_SENSOR_PATH, None)

    # Search Indoor Data
    if indoor_data:
        for timestamp, record in indoor_data.items():
            for key, value in record.items():
                if query_lower in str(key).lower() or query_lower in str(value).lower():
                    results.append(f"Indoor [{timestamp}] - {key}: {value}")

    # Search Outdoor Data
    if outdoor_data:
        for timestamp, record in outdoor_data.items():
            for key, value in record.items():
                if query_lower in str(key).lower() or query_lower in str(value).lower():
                    results.append(f"Outdoor [{timestamp}] - {key}: {value}")

    if not results:
        results.append("No matching results found.")

    return results


def fetch_latest_sensor_data() -> SensorData:
    """Fetches the latest data for indoor and outdoor sensors from Firebase."""
    indoor_data = FBconn.get(INDOOR_SENSOR_PATH, None)
    outdoor_data = FBconn.get(OUTDOOR_SENSOR_PATH, None)
    latest_indoor = {}
    if indoor_data:
        # Assuming the keys are timestamps or some ordered identifier
        latest_key = max(indoor_data.keys(), key=lambda k: indoor_data[k].get('Time', ''))
        latest_indoor = indoor_data[latest_key]

    latest_outdoor = {}
    if outdoor_data:
        latest_key = max(outdoor_data.keys(), key=lambda k: outdoor_data[k].get('Time', ''))
        latest_outdoor = outdoor_data[latest_key]

    return {
        "Indoor Sensor": latest_indoor,
        "Outdoor Sensor": latest_outdoor,
    }

def generate_sensor_statistics(selected_sensor: str, sensor_data: SensorData) -> widgets.HTML:
    """Generates and returns an HTML widget displaying statistics for the selected sensor."""
    if selected_sensor in sensor_data and sensor_data[selected_sensor]:
        data = sensor_data[selected_sensor]
        stats_text = f"""
            <div style='display: flex; justify-content: space-between;'>
                <div style='flex: 1; margin-right: 20px;'>
                    <b>Sensor:</b> {selected_sensor}<br>
        """
        for key, value in data.items():
            stats_text += f"<b>{key}:</b> {value}<br>"
        stats_text += """
                </div>
            </div>
        """
        return create_html_widget(stats_text)
    else:
        return create_html_widget(f"No data available for {selected_sensor}.")

def fetch_historical_sensor_data(sensor_path: str) -> HistoricalSensorData:
    """Fetches all historical data for a given sensor from Firebase."""
    data = FBconn.get(sensor_path, None)
    if data:
        historical_data: HistoricalSensorData = {"Timestamp": [], "Humidity": [], "Temperature": [], "Pressure": [], "Distance": []}
        for entry in data.values():
            historical_data["Timestamp"].append(entry.get("Time", ""))
            historical_data["Humidity"].append(entry.get("Humidity"))
            historical_data["Temperature"].append(entry.get("Temperature"))
            historical_data["Pressure"].append(entry.get("Pressure"))
            historical_data["Distance"].append(entry.get("Distance"))
            if sensor_path == OUTDOOR_SENSOR_PATH:
                if "DLIGHT" in entry:
                    if "DLIGHT" not in historical_data:
                        historical_data["DLIGHT"] = []
                    historical_data["DLIGHT"].append(entry["DLIGHT"])
        return historical_data
    else:
        return {"Timestamp": [], "Humidity": [], "Temperature": [], "Pressure": [], "Distance": []}

def create_historical_plots(sensor_name: str) -> widgets.Output:
    """Creates historical plots for the selected sensor with improved formatting."""
    if sensor_name == "Indoor Sensor":
        historical_data = fetch_historical_sensor_data(INDOOR_SENSOR_PATH)
        attributes = ["Humidity", "Temperature", "Pressure", "Distance"]
        colors = ['b', 'r', 'g', 'm']  # Blue, red, green, magenta
    elif sensor_name == "Outdoor Sensor":
        historical_data = fetch_historical_sensor_data(OUTDOOR_SENSOR_PATH)
        attributes = ["Humidity", "Temperature", "Pressure", "Distance", "DLIGHT"]
        colors = ['b', 'r', 'g', 'm', 'orange']  # Blue, red, green, magenta, orange
    else:
        out = widgets.Output()
        with out:
            print(f"Invalid sensor name: {sensor_name}")
        return out

    if not historical_data["Timestamp"]:
        out = widgets.Output()
        with out:
            print(f"No historical data available for {sensor_name}")
        return out

    out = widgets.Output()
    with out:
        num_plots = len(attributes)
        # Calculate optimal grid layout
        rows = (num_plots + 1) // 2  # Ceiling division to ensure enough rows
        cols = 2 if num_plots > 1 else 1

        plt.figure(figsize=(14, 5 * rows))

        times = historical_data["Timestamp"]
        # Convert timestamps to datetime if they aren't already
        if times and not isinstance(times[0], datetime.datetime):
            try:
                # Attempt to convert timestamps to datetime objects
                times = [datetime.datetime.strptime(t, "%H:%M:%S") if isinstance(t, str) else
                         datetime.datetime.fromtimestamp(t) if isinstance(t, (int, float)) else t
                         for t in times]
            except ValueError:
                # If conversion fails, we'll continue with original timestamps
                pass

        def format_axis(ax, times, ydata, color, ylabel):
            """Helper function to format each subplot axis."""
            if not ydata:
                ax.text(0.5, 0.5, f"No data for {ylabel}",
                        horizontalalignment='center', verticalalignment='center')
                ax.set_title(f"{ylabel} over Time ({sensor_name})")
                return

            ax.plot(times, ydata, marker='o', color=color, label=ylabel)
            ax.set_title(f"{ylabel} over Time ({sensor_name})")
            ax.set_xlabel("Time")
            ax.set_ylabel(ylabel)

            # Format time axis if we have datetime objects
            if times and isinstance(times[0], datetime.datetime):
                ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
                ax.xaxis.set_major_locator(mdates.AutoDateLocator())
                plt.setp(ax.get_xticklabels(), rotation=45, ha='right')

            # Set y-axis limits
            valid_ydata = [y for y in ydata if y is not None]
            if valid_ydata:
                y_min = min(valid_ydata)
                y_max = max(valid_ydata)
                if y_min != y_max:
                    y_range = y_max - y_min
                    ax.set_ylim(y_min - 0.1 * y_range, y_max + 0.1 * y_range)

            ax.grid(True)
            ax.legend()

        # Create and format each subplot
        for i, (attribute, color) in enumerate(zip(attributes, colors)):
            ax = plt.subplot(rows, cols, i + 1)
            y_data = historical_data.get(attribute, [])
            format_axis(ax, times, y_data, color, attribute)

        plt.tight_layout()
        plt.show()
    return out

def update_statistics_display(change) -> None:
    """Updates the statistics display based on the selected sensor in the dropdown."""
    selected_sensor = change['new']
    latest_data = fetch_latest_sensor_data()
    stats_display.value = generate_sensor_statistics(selected_sensor, latest_data).value

    # Clear the output widget and display the new figures
    stats_plots.clear_output(wait=True)
    with stats_plots:
        out = create_historical_plots(selected_sensor)
        display(out)

# --- Main Menu Screen Widgets ---
main_menu_title = create_html_widget(
    "<h1 style='color:#3498db; text-align:center; margin-bottom:30px;'>Robotics Lab Control Center</h1>",
    margin='0 0 20px 0'
)

# Create large, attractive menu buttons
manager_btn = create_button(
    "System Manager",
    button_style='info',
    width='280px',
    height='80px',
    margin='10px',
    font_weight='bold',
    box_shadow=BOX_SHADOW
)

statistics_btn = create_button(
    "Sensor Statistics",
    button_style='success',
    width='280px',
    height='80px',
    margin='10px',
    font_weight='bold',
    box_shadow=BOX_SHADOW
)

search_btn = create_button(
    "Search Query",
    button_style='warning',
    width='280px',
    height='80px',
    margin='10px',
    font_weight='bold',
    box_shadow=BOX_SHADOW
)

optimization_btn = create_button(
    "Optimization Race",
    button_style='danger',
    width='280px',
    height='80px',
    margin='10px',
    font_weight='bold',
    box_shadow=BOX_SHADOW
)

# Create dashboard summary for main menu
dashboard_summary = create_html_widget(
    f"""
    <div style='padding:15px; background-color:white; border-radius:10px; box-shadow:{BOX_SHADOW}'>
        <h3 style='color:{DARKER_BLUE_COLOR}; text-align:center; margin-bottom:15px;'>Dashboard Summary</h3>
        <div style='display:flex; justify-content:space-between; flex-wrap:wrap;'>
            <div style='flex:1; min-width:250px; padding:10px; margin:5px; background-color:{LIGHTER_BLUE_COLOR}; border-radius:8px;'>
                <h4 style='color:{DARKER_BLUE_COLOR};'>Sensors</h4>
                <p><span style='font-weight:bold;'>Indoor:</span> Available</p>
                <p><span style='font-weight:bold;'>Outdoor:</span> Available</p>
            </div>
            <div style='flex:1; min-width:250px; padding:10px, margin:5px; background-color:{LIGHTER_BLUE_COLOR}; border-radius:8px;'>
                <h4 style='color:{DARKER_BLUE_COLOR};'>Daily Tasks</h4>
                <p><span style='font-weight:bold;'>{len(daily_tasks)}</span> tasks pending</p>
            </div>
            <div style='flex:1; min-width:250px; padding:10px, margin:5px; background-color:{LIGHTER_BLUE_COLOR}; border-radius:8px;'>
                <h4 style='color:{DARKER_BLUE_COLOR};'>Optimization Race</h4>
                <p>Leader: <span style='font-weight:bold;'>Charlie</span> with <span style='color:{ORANGE_COLOR}; font-weight:bold;'>1800</span> points</p>
            </div>
        </div>
    </div>
    """,
    margin='20px 0'
)

# Arrange menu buttons in a grid layout
top_row = create_hbox([manager_btn, statistics_btn], justify_content='center')
bottom_row = create_hbox([search_btn, optimization_btn], justify_content='center')
menu_buttons = create_vbox(
    [top_row, bottom_row],
    border='none',
    background_color='transparent',
    padding='10px'
)

# Welcome message with date/time
import datetime
current_time = datetime.datetime.now(local_tz).strftime("%A, %B %d, %Y %I:%M %p")
welcome_msg = create_html_widget(
    f"<div style='text-align:center; margin:10px 0 30px 0;'><h3>Welcome to Robotics Lab Control Center</h3><p>{current_time}</p></div>"
)

# Main menu container
main_menu_container = create_vbox(
    [
        main_menu_title,
        welcome_msg,
        dashboard_summary,
        menu_buttons
    ],
    background_color=LIGHTER_BLUE_COLOR,
    border=f'2px solid {DARKER_BLUE_COLOR}',
    box_shadow=BOX_SHADOW,
    width='800px',
    margin='0 auto'
)

# --- System Manager Screen Widgets ---
manager_title = create_html_widget("<h2 style='color:#3498db;'>Robotics Lab System Manager Dashboard</h2>")
production_status_title = create_html_widget("<h3>Sensor Status</h3>")
sensor_status_display = create_html_widget("") # To display indoor/outdoor status
tasks_title = create_html_widget("<h3>Daily Optimization Tasks</h3>")
tasks_list = create_html_widget("<ul style='list-style-type:disc; padding-left:20px;'>" + "".join([f"<li>{task}</li>" for task in daily_tasks]) + "</ul>")
leaderboard_title = create_html_widget("<h3>Engineer Optimization Race Leaderboard</h3>")
leaderboard_items = [f"<li><span style='font-weight:bold;'>{engineer}</span>: <span style='color:{ORANGE_COLOR};'>{performance['Points']}</span> Points, <span style='color:{ORANGE_COLOR};'>{performance['Improvements Completed']}</span> Improvements</span></li>"
                    for engineer, performance in engineers_performance.items()]
leaderboard = create_html_widget("<ol style='padding-left:20px;'>" + "".join(leaderboard_items) + "</ol>")

manager_back_btn = create_button("Back to Main Menu", button_style='info')
manager_output = create_output()
manager_container = create_vbox(
    [
        manager_title,
        production_status_title,
        widgets.VBox([sensor_status_display], layout=widgets.Layout(border=f'1px solid {LIGHT_GREY_BORDER}', border_radius='8px', padding='15px', margin_bottom='15px',
                                                                        background_color=WHITE_COLOR)),
        tasks_title,
        tasks_list,
        leaderboard_title,
        leaderboard,
        manager_back_btn,
        manager_output
    ]
)

def update_sensor_status() -> None:
    """Updates the sensor status display on the manager screen."""
    latest_data = fetch_latest_sensor_data()
    indoor_status = "Available" if latest_data.get("Indoor Sensor") else "Not Available"
    outdoor_status = "Available" if latest_data.get("Outdoor Sensor") else "Not Available"
    status_text = f"""
        <b>Indoor Sensor:</b> <span style='font-weight:bold;'>{indoor_status}</span><br>
        <b>Outdoor Sensor:</b> <span style='font-weight:bold;'>{outdoor_status}</span>
    """
    sensor_status_display.value = status_text

# --- Search Query Screen Widgets ---
query_title = create_html_widget("<h2 style='color:#3498db;'>Search Query</h2>")
query_input = widgets.Text(
    description="Enter your query:",
    style={'description_width': 'initial'},  # This prevents truncation
    layout=widgets.Layout(
        width='600px',
        height='40px',
        border_radius=BORDER_RADIUS,
        padding='10px',
        font_size='16px'
    )
)
search_button = create_button("Search")
query_results = create_output(layout=widgets.Layout(height='300px', border=f'1px solid {LIGHT_GREY_BORDER}', border_radius='8px', overflow='auto', background_color='rgba(255,255,255,0.8)', padding='15px'))
query_back_btn = create_button("Back to Main Menu", button_style='info')
query_container = create_vbox(
    [
        query_title,
        query_input,
        search_button,
        query_results,
        query_back_btn
    ]
)

# --- Statistics Screen Widgets ---
stats_title = create_html_widget("<h2 style='color:#2c3e50; text-align:center;'>Sensor Statistics</h2>")
stats_display = create_html_widget(
    "",
    border_radius='8px',
    padding='15px',
    margin_bottom='15px',
    background_color='rgba(255,255,255,0.9)',
    box_shadow=BOX_SHADOW
)
sensor_selector = create_dropdown(
    options=["Indoor Sensor", "Outdoor Sensor"],
    description="<b style='color:#2c3e50;'>Select Sensor:</b>"
)
stats_back_btn = create_button("Back to Main Menu", button_style='info', layout=widgets.Layout(width='200px', height='50px', border_radius='10px', font_size='16px', margin='20px auto'))

# Create a dedicated output widget for plots with proper layout
stats_plots = widgets.Output(layout=widgets.Layout(
    min_height='600px',  # Increased height to accommodate the graphs.  Adjust as needed.
    width='100%',
    border=f'1px solid {LIGHT_GREY_BORDER}',
    border_radius='8px',
    background_color='rgba(240,248,255,0.8)',
    box_shadow=BOX_SHADOW,
    padding='20px',
    overflow='visible'
))

stats_container = create_vbox(
    [
        stats_title,
        sensor_selector,
        stats_display,
        stats_plots,
        stats_back_btn
    ],
    layout=widgets.Layout(padding='30px', border=f'2px solid {DARKER_BLUE_COLOR}', border_radius='15px', margin='20px auto', background_color=LIGHTER_BLUE_COLOR, box_shadow=BOX_SHADOW)
)

# --- Optimization Race Screen Widgets ---
optimization_title = create_html_widget("<h2 style='color:#3498db;'>Optimization Race</h2>")
optimization_leaderboard_title = create_html_widget("<h3>Current Standings</h3>")
optimization_leaderboard = create_html_widget("")
optimization_tasks_title = create_html_widget("<h3>Your Daily Tasks</h3>")
optimization_tasks_list = create_html_widget("<ul style='list-style-type:disc; padding-left:20px;'>" + "".join([f"<li>{task}</li>" for task in daily_tasks]) + "</ul>")
optimization_back_btn = create_button("Back to Main Menu", button_style='info')
optimization_output = create_output()
optimization_container = create_vbox(
    [
        optimization_title,
        optimization_leaderboard_title,
        optimization_leaderboard,
        optimization_tasks_title,
        optimization_tasks_list,
        optimization_back_btn,
        optimization_output
    ]
)

def update_optimization_leaderboard() -> None:
    """Updates the leaderboard displayed on the optimization screen."""
    sorted_engineers = sorted(engineers_performance.items(), key=lambda item: item[1]['Points'], reverse=True)
    leaderboard_items = []
    medal_icons = {
        0: "🥇",
        1: "🥈",
        2: "🥉"
    }
    for i, (engineer, performance) in enumerate(sorted_engineers):
        medal = medal_icons.get(i, "")
        leaderboard_items.append(f"<li>{medal} <span style='font-weight:bold;'>{engineer}</span>: <span style='color:{ORANGE_COLOR};'>{performance['Points']}</span> Points, <span style='color:{ORANGE_COLOR};'>{performance['Improvements Completed']}</span> Improvements</span></li>")
    optimization_leaderboard.value = "<ol style='padding-left:20px;'>" + "".join(leaderboard_items) + "</ol>"

#--- ScreenManagement Functions ---

current_screen = "main_menu"
displayed_screen = widgets.VBox([])

def show_main_menu() -> None:
    """Displays the main menu screen."""
    global current_screen, displayed_screen
    current_screen = "main_menu"
    clear_output(wait=True)
    displayed_screen.children = [main_menu_container]
    display(displayed_screen)

def show_manager_screen() -> None:
    """Displays the manager screen."""
    global current_screen, displayed_screen
    current_screen = "manager"
    clear_output(wait=True)
    displayed_screen.children = [manager_container]
    display(displayed_screen)
    update_sensor_status()

def show_query_screen() -> None:
    """Displays the query screen."""
    global current_screen, displayed_screen
    current_screen = "query"
    clear_output(wait=True)
    displayed_screen.children = [query_container]
    display(displayed_screen)

def show_statistics_screen() -> None:
    """Displays the statistics screen."""
    global current_screen, displayed_screen
    current_screen = "statistics"
    clear_output(wait=True)
    displayed_screen.children = [stats_container]
    display(displayed_screen)

    # Initialize with the first sensor's data
    initial_sensor = sensor_selector.value
    latest_data = fetch_latest_sensor_data()
    stats_display.value = generate_sensor_statistics(initial_sensor, latest_data).value

    # Clear and update plots
    stats_plots.clear_output(wait=True)
    with stats_plots:
        out = create_historical_plots(initial_sensor)
        display(out)

def show_optimization_screen() -> None:
    """Displays the optimization screen."""
    global current_screen, displayed_screen
    current_screen = "optimization"
    clear_output(wait=True)
    displayed_screen.children = [optimization_container]
    display(displayed_screen)
    update_optimization_leaderboard()

def update_screen() -> None:
    """Updates the displayed screen based on the current screen variable."""
    if current_screen == "main_menu":
        show_main_menu()
    elif current_screen == "manager":
        show_manager_screen()
    elif current_screen == "query":
        show_query_screen()
    elif current_screen == "statistics":
        show_statistics_screen()
    elif current_screen == "optimization":
        show_optimization_screen()

# --- Event Handlers ---

def handle_main_menu_navigation(button: widgets.Button) -> None:
    """Handles navigation from the main menu screen."""
    if button.description == "System Manager":
        show_manager_screen()
    elif button.description == "Sensor Statistics":
        show_statistics_screen()
    elif button.description == "Search Query":
        show_query_screen()
    elif button.description == "Optimization Race":
        show_optimization_screen()

def handle_back_to_menu(button: widgets.Button) -> None:
    """Handles back to main menu navigation from any screen."""
    show_main_menu()

def handle_search(button: widgets.Button) -> None:
    """Handles the search query and displays results."""
    query = query_input.value
    results = perform_search(query)
    query_results.clear_output(wait=True)
    with query_results:
        display(widgets.HTML("<br>".join(results)))

# --- Widget Event Assignments ---
# Main menu navigation
manager_btn.on_click(handle_main_menu_navigation)
statistics_btn.on_click(handle_main_menu_navigation)
search_btn.on_click(handle_main_menu_navigation)
optimization_btn.on_click(handle_main_menu_navigation)

# Back buttons
manager_back_btn.on_click(handle_back_to_menu)
stats_back_btn.on_click(handle_back_to_menu)
query_back_btn.on_click(handle_back_to_menu)
optimization_back_btn.on_click(handle_back_to_menu)

# Search functionality
search_button.on_click(handle_search)

# Sensor selector in statistics screen
sensor_selector.observe(update_statistics_display, names='value')

# --- Initial Display ---
show_main_menu()
